### Importing Libraries

This cell imports the necessary libraries for the project. `keras` is a high-level neural networks API, and `keras_nlp` provides additional tools and functionalities for natural language processing tasks using Keras.


In [1]:
import keras
import keras_nlp
import tensorflow

2024-08-29 18:01:15.929029: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 18:01:15.944717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 18:01:15.963838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 18:01:15.969620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 18:01:15.983677: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# import os

# os.environ["KERAS_BACKEND"] = "jax"  
# # Avoid memory fragmentation on JAX backend.
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import json

# Initialize an empty list to hold the processed data.
data = []

# Open and read the JSON file line by line.
with open('/project/data/combined_dataset.json') as file:
    for line in file:
        features = json.loads(line)
        
        # Filter out examples without "Context".
        if not features.get("Context"):
            continue
        
        # Format the example as a string.
        template = "Instruction:\n{Context}\n\nResponse:\n{Response}"
        formatted_example = template.format(**features)
        
        # Append the formatted example to the data list.
        data.append(formatted_example)

# Now data contains a list of formatted strings.


In [ ]:
import os

# Set Kaggle API credentials
os.environ["KAGGLE_USERNAME"] = "rogerkorantenng"
os.environ["KAGGLE_KEY"] = "9a33b6e88bcb6058b1281d777fa6808d"


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

In [ ]:
# Define the template with placeholders for 'instruction' and 'response'
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Create the prompt by formatting the template with actual data
prompt = template.format(
    instruction="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    response="",
)

# Assuming gemma_lm is a language model that you're using to generate text
print(gemma_lm.generate(prompt, max_length=256))


In [ ]:
# gemma_lm.save("gemma_model.h5")


In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=10)
gemma_lm.summary()

In [ ]:
import os
import tensorflow as tf
import keras_nlp
import keras
import json

# Set Kaggle API credentials
os.environ["KAGGLE_USERNAME"] = "rogerkorantenng"
os.environ["KAGGLE_KEY"] = "9a33b6e88bcb6058b1281d777fa6808d"

In [ ]:
def get_compiled_model():
    gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
    gemma_lm.summary()

    gemma_lm.backbone.enable_lora(rank=2)
    gemma_lm.summary()
    
    # Set the sequence length to 128 before using the model.
    gemma_lm.preprocessor.sequence_length = 128
    
    # Use AdamW (a common optimizer for transformer models).
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01,
    )
    
    # Exclude layernorm and bias terms from decay.
    optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
    
    gemma_lm.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    
    return gemma_lm
print(gemma_lm.)

print(gemma_lm.summary())


    

In [ ]:
def get_dataset():
    # Initialize an empty list to hold the processed data.
    data = []
    
    # Open and read the JSON file line by line.
    with open('/project/data/HealthCareMagic-100k-en.jsonl') as file:
        for line in file:
            features = json.loads(line)
            
            # Filter out examples without "Context".
            if not features.get("Context"):
                continue
            
            # Format the example as a string.
            template = "Instruction:\n{Context}\n\nResponse:\n{Response}"
            formatted_example = template.format(**features)
            
            # Append the formatted example to the data list.
            data.append(formatted_example)
    
    return data 

In [ ]:
model = get_compiled_model()

# Get the dataset outside the strategy scope.
data = get_dataset()

# Fit the model using the data.
model.fit(data, epochs=2, batch_size=0, verbose=1)

In [2]:
print('Hello')

Hello
